<a href="https://colab.research.google.com/github/vinayak15/NLP-Project-2/blob/main/Proejct_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/drive/MyDrive/NLP/Project2/joint_model/

/content/drive/MyDrive/NLP/Project2/joint_model


In [ ]:
!git clone https://github.com/luomancs/joint_model

In [ ]:
!pip install -r requirements.txt

In [3]:
!pip uninstall apex

Found existing installation: apex 0.9.10.dev0
Uninstalling apex-0.9.10.dev0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/apex-0.9.10.dev0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/apex/*
Proceed (y/n)? y
  Successfully uninstalled apex-0.9.10.dev0


In [ ]:
import json, os
import random
from tqdm import tqdm
import numpy as np
import string 
import math

from common.utils_plus import *

def hotpot(read_path, save_path,mode="train"):
    posi_para = []
    neg_para = []
    data = json_plus.load(read_path)
    for i,item in enumerate(tqdm(data)):
        question = item['question'] 
        answer = item['answer']
        supporting_title = {}
        for sp in item['supporting_facts']:
            title = sp[0]
            idx = sp[1]
            if title not in supporting_title:
                supporting_title[title] = [idx]
            else:
                supporting_title[title].append(idx)
        
        for para in item['context']:
            title = para[0]
            sents_label = [-1]*(len(para[1]))
            para_label = -1
            if  title in supporting_title:
                para_label = 1
                for idx in supporting_title[title]:
                    if  idx < len(para[1]):
                        sents_label[idx] = 1
            
            datum = {
                'para':"</s> ".join([question]+para[1][:9]),
                'para_label':para_label,
                'sents_label':sents_label[:9],
                'answer_label': answer
            }
            if title in supporting_title:
                posi_para.append(datum)
            else:
                neg_para.append(datum)
              

    if mode=="train":
        neg_para = random.sample(neg_para, len(posi_para))
    all_samples = posi_para+neg_para
    print(all_samples)
    print(len(all_samples))
    jsonline_plus.dump(all_samples, save_path)

translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))

def normalize(str1):
    str2 = str1.translate(translator)
    str2 = str2.lower().replace(" ","")
    return str2

In [ ]:
hotpot('/content/drive/MyDrive/NLP/Project2/dev.json', '/content/drive/MyDrive/NLP/Project2/joint_model/dataset/para_training/dev.jsonl',mode="dev")

100%|██████████| 7405/7405 [00:00<00:00, 19358.71it/s]
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
!python model_train/train_para.py \
--model_name_or_path roberta-base \
--tokenizer_name roberta-base \
--task_name para \
--data_dir dataset/para_training/ \
--max_seq_length 512 \
--output_dir path_to_save_model \
--do_train \
--do_eval \
--overwrite_output_dir \
--per_device_train_batch_size 4 \
--gradient_accumulation_steps 2 \
--per_device_eval_batch_size 32 \
--learning_rate 1e-5 \
--num_train_epochs 5 \
--save_steps 10000 \
--logging_steps 10000 \
--use_sent_loss \

11/27/2021 05:07:09 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
11/27/2021 05:07:09 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=2,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=1e-05,
length_column_name=length,
load_best_model